In [1]:
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing import image
import numpy as np
import cv2
import os

In [2]:
# Load model (without top layer)
model = Xception(weights='imagenet', include_top=False, pooling='avg')

def extract_frames(video_path, max_frames=10):
    cap = cv2.VideoCapture(video_path)
    frames = []
    count = 0
    while cap.isOpened() and count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        # Resize + convert to array
        frame = cv2.resize(frame, (299, 299))
        img_array = image.img_to_array(frame)
        img_array = preprocess_input(img_array)
        frames.append(img_array)
        count += 1
    cap.release()
    return np.array(frames)


In [3]:
def is_deepfake(video_path):
    frames = extract_frames(video_path)
    if len(frames) == 0:
        return "Invalid or corrupted video"

    # Extract features using Xception
    features = model.predict(frames, verbose=0)
    avg_feature = np.mean(features, axis=0)

    # Dummy classifier: If average feature value > threshold → Fake
    # (In real-world, train an actual classifier like SVM or NN)
    fake_score = np.mean(avg_feature)
    print(f"Fake Score: {fake_score:.4f}")
    return "FAKE" if fake_score > 0.55 else "REAL"

In [4]:
video_path = 'sample_video.mp4'
result = is_deepfake(video_path)
print("Prediction:", result)


Fake Score: 0.1594
Prediction: REAL
